In [32]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.models import Model

# Load pre-trained MobileNetV2
mobilenet_base = tf.keras.applications.MobileNetV3Large(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)



In [33]:
mobilenet_base.summary()

Model: "MobileNetV3Large"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_13            │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_8 (Rescaling)   │ (None, 224, 224, 3)    │              0 │ input_layer_13[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv (Conv2D)             │ (None, 112, 112, 16)   │            432 │ rescaling_8[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv_bn                   │ (None, 112, 112, 16)   │             64 │ conv[0][0]             │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_160            │ (None, 112, 112, 16)   │              0 │ conv_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 112, 112, 16)   │            144 │ activation_160[0][0]   │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 16)   │             64 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_152 (ReLU)          │ (None, 112, 112, 16)   │              0 │ expanded_conv_depthwi… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 112, 112, 16)   │            256 │ re_lu_152[0][0]        │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_bn  │ (None, 112, 112, 16)   │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_add (Add)   │ (None, 112, 112, 16)   │              0 │ activation_160[0][0],  │
│                           │                        │                │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_1_expand    │ (None, 112, 112, 64)   │          1,024 │ expanded_conv_add[0][… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_1_expand_bn │ (None, 112, 112, 64)   │            256 │ expanded_conv_1_expan… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_153 (ReLU)          │ (None, 112, 112, 64)   │              0 │ expanded_conv_1_expan… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_1_depth

 Total params: 2,996,352 (11.43 MB)

 Trainable params: 2,971,952 (11.34 MB)

 Non-trainable params: 24,400 (95.31 KB)

# MobileNet Architecture and Key Concepts

## **What is MobileNet?**
MobileNet is a family of lightweight deep learning models designed for mobile and embedded devices. It achieves efficiency and low computational cost through techniques like **depthwise separable convolutions**. These models are ideal for tasks requiring real-time predictions in resource-constrained environments.

### **MobileNet Versions**
1. **MobileNetV1**:
   - Introduced depthwise separable convolutions to reduce computations.
   - Lightweight but less accurate compared to newer versions.

2. **MobileNetV2**:
   - Added **inverted residual blocks** and **linear bottlenecks** for better feature extraction and efficiency.

3. **MobileNetV3**:
   - Combines ideas from **MobileNetV2** and **Neural Architecture Search (NAS)**.
   - Includes:
     - **Squeeze-and-Excitation (SE) Blocks** for channel-wise attention.
     - **Hard-Swish Activation** for improved gradients and efficiency.
     - **Small** and **Large** variants tailored for different resource requirements.

---

## **Key Architectural Features**

### **Depthwise Separable Convolutions**
- **Depthwise Convolution**: Applies one filter per input channel to reduce computation.
- **Pointwise Convolution**: Uses 1x1 convolutions to combine outputs from depthwise convolutions.

### **Inverted Residuals and Linear Bottlenecks** (MobileNetV2 and V3)
- **Inverted Residuals**: Shortcut connections between thin bottleneck layers.
- **Linear Bottleneck**: Prevents information loss by using linear activation instead of ReLU in bottleneck layers.

### **Squeeze-and-Excitation (SE) Blocks**
- Channel-wise attention mechanism that learns to focus on important features while ignoring irrelevant ones.

### **Hard-Swish Activation**
- A computationally efficient alternative to standard Swish activation, improving model performance with minimal cost.

---

## **Weights and Layers in MobileNet**

### **How Weights Work in MobileNet**
- MobileNet uses pre-trained weights when loaded with `weights='imagenet'`. These weights are optimized for recognizing features in ImageNet images, such as edges, textures, and shapes.
- The pre-trained weights are found in trainable layers (e.g., convolutional layers).

### **Accessing Weights in Layers**
1. **Inspecting Layers**:
   - Use a loop to inspect all layers in the model and identify which ones contain weights:
     ```python
     for i, layer in enumerate(base_model.layers):
         print(i, layer.name, type(layer), len(layer.weights))
     ```

2. **Extracting Weights**:
   - Locate a specific layer and retrieve its weights and biases:
     ```python
     conv_layer = base_model.layers[1]  # Replace with correct index
     weights, biases = conv_layer.get_weights()
     print("Weights shape:", weights.shape)
     print("Biases shape:", biases.shape)
     ```

3. **Handling Empty Weights**:
   - Layers like input or preprocessing layers do not have trainable weights, so `get_weights()` may return an empty list.
   - Ensure the model is initialized by passing a dummy input:
     ```python
     dummy_input = tf.random.normal([1, 224, 224, 3])
     base_model(dummy_input)  # Forward pass to build the model
     ```

### **Impact of Replacing Non-Trainable Layers**
- Non-trainable layers, such as input or activation layers, do not significantly affect training as they lack learnable parameters.
- Replacing them is generally safe but ensure downstream layers can handle the changes (e.g., modified input shapes).

### **When to Modify Weights**
- Modify trainable layers when adapting the model to new tasks (e.g., handling 5-channel input: RGB + mask + texture).
- Ensure new weights for additional channels are initialized properly (e.g., using random initialization) and fine-tuned during training.

---

## **Adapting MobileNet for 5-Channel Input**

### **Problem**
Standard MobileNet expects 3-channel input (RGB). To handle 5-channel input (e.g., RGB + mask + texture):
- Modify the first convolutional layer to accept 5 channels.

### **Steps to Modify**
1. **Inspect the Model**:
   - Identify the first convolutional layer with weights.

2. **Extend Weights**:
   - Add random weights for the additional channels:
     ```python
     new_weights = tf.concat([weights, tf.random.normal([3, 3, 2, weights.shape[-1]])], axis=2)
     ```

3. **Replace the Layer**:
   - Create a new input layer and attach the modified convolutional layer to the model.

4. **Fine-Tune the Model**:
   - Train the model on your dataset to adapt it to the new input format.

---

## **Conclusion**
- MobileNet is highly flexible and efficient, making it ideal for mobile and embedded applications.
- Pre-trained weights can be adapted to new tasks by modifying the architecture.
- Replacing non-trainable layers has minimal impact, but ensure compatibility with downstream layers.
- When adding new input channels, extend the first layer's weights and fine-tune the model for optimal performance.



In [25]:
original_conv_weights = mobilenet_base.layers[2].get_weights()
original_conv_weights[0][]

AttributeError: 'Conv2D' object has no attribute 'shape'

In [55]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.models import Model

# Load MobileNetV3Large
mobilenet_base = MobileNetV3Large(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

# Get the weights of the first convolutional layer
original_conv_weights = mobilenet_base.layers[2].get_weights()  # Usually, it's the second layer but check it via mobilenet_base.summary()

# Ensure there are no biases
if len(original_conv_weights) == 1:
    print("No biases detected. Proceeding with kernel weight adjustment.")
kernel_weights = original_conv_weights[0]  # Shape: (filter_height, filter_width, input_channels, output_channels)

# Adjust kernel weights (duplicate or average the weights for new channels)
new_kernel_weights = tf.concat(
    [kernel_weights, kernel_weights[:, :, :2, :]],  # Duplicate first two channels
    axis=2
)  # Shape: (filter_height, filter_width, 5, output_channels)

# Define a new input layer with 5 channels
input_layer = Input(shape=(224, 224, 5))

# Define a new Conv2D layer with adjusted weights
conv_layer = Conv2D(
    filters=mobilenet_base.layers[2].filters,
    kernel_size=mobilenet_base.layers[2].kernel_size,
    strides=mobilenet_base.layers[2].strides,
    padding=mobilenet_base.layers[2].padding,
    use_bias=False  # Explicitly specify no bias
)(input_layer)

# Create a temporary model to set weights
temp_model = Model(inputs=input_layer, outputs=conv_layer)
temp_model.layers[-1].set_weights([new_kernel_weights])
temp_model.summary()
# Now, let's rebuild the model
# Starting from the output of our custom conv layer
x = temp_model.output

# Get the output from the first BN layer which is the next layer after Conv2D in MobileNetV3
x = mobilenet_base.layers[3](x)

# Build the remaining layers sequentially
for index in range(0, len(mobilenet_base.layers[3:])):
    x = mobilenet_base.layers[3+index](x)

# Create the final model
model = Model(inputs=input_layer, outputs=x)

# Summary to verify the changes
model.summary()


No biases detected. Proceeding with kernel weight adjustment.


Model: "functional_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_47 (InputLayer)          │ (None, 224, 224, 5)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_17 (Conv2D)                   │ (None, 112, 112, 16)        │             720 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 720 (2.81 KB)

 Trainable params: 720 (2.81 KB)

 Non-trainable params: 0 (0.00 B)

In [56]:
mobilenet_base

TypeError: 'Functional' object is not subscriptable

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.models import Model

# Load MobileNetV3Large
mobilenet_base = MobileNetV3Large(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

# Get the weights of the first convolutional layer
original_conv_weights = mobilenet_base.layers[2].get_weights()  # Usually, it's the second layer but check it via mobilenet_base.summary()

# Ensure there are no biases
if len(original_conv_weights) == 1:
    print("No biases detected. Proceeding with kernel weight adjustment.")
kernel_weights = original_conv_weights[0]  # Shape: (filter_height, filter_width, input_channels, output_channels)

# Adjust kernel weights (duplicate or average the weights for new channels)
new_kernel_weights = tf.concat(
    [kernel_weights, kernel_weights[:, :, :2, :]],  # Duplicate first two channels
    axis=2
)  # Shape: (filter_height, filter_width, 5, output_channels)

# Define a new input layer with 5 channels
input_layer = Input(shape=(224, 224, 5))

# Define a new Conv2D layer with adjusted weights
conv_layer = Conv2D(
    filters=mobilenet_base.layers[2].filters,
    kernel_size=mobilenet_base.layers[2].kernel_size,
    strides=mobilenet_base.layers[2].strides,
    padding=mobilenet_base.layers[2].padding,
    use_bias=False  # Explicitly specify no bias
)(input_layer)

# Create a temporary model to set weights
temp_model = Model(inputs=input_layer, outputs=conv_layer)
temp_model.layers[-1].set_weights([new_kernel_weights])
temp_model.summary()
# Now, let's rebuild the model
# Starting from the output of our custom conv layer
x = temp_model.output

# Get the output from the first BN layer which is the next layer after Conv2D in MobileNetV3
x = mobilenet_base.layers[3](x)

# Build the remaining layers sequentially
for index in range(0, len(mobilenet_base.layers[3:])):
    x = mobilenet_base.layers[3+index](x)

# Create the final model
model = Model(inputs=input_layer, outputs=x)

# Summary to verify the changes
model.summary()


In [62]:
import torch
from torchvision import models
from torchvision.models import MobileNet_V3_Large_Weights
from torch.nn import Conv2d

# Load the pre-trained MobileNetV3 model
mobilenet = models.mobilenet_v3_large(weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1)

# Extract the first convolutional layer properties
first_conv_layer = mobilenet.features[0][0]

# Create a new Conv2d layer with 5 input channels instead of 3
new_first_conv_layer = Conv2d(
    in_channels=5,
    out_channels=first_conv_layer.out_channels,
    kernel_size=first_conv_layer.kernel_size,
    stride=first_conv_layer.stride,
    padding=first_conv_layer.padding,
    bias=(first_conv_layer.bias is not None)  # Preserve the use of bias if it was used
)

# Initialize the new convolutional layer weights
with torch.no_grad():
    # Copy weights from the first three channels
    new_first_conv_layer.weight[:, :3, :, :] = first_conv_layer.weight.clone()

    # Initialize the weights for the two new channels by averaging the original three channels
    new_channel_weights = first_conv_layer.weight.mean(dim=1, keepdim=True).expand(-1, 2, -1, -1)
    new_first_conv_layer.weight[:, 3:5, :, :] = new_channel_weights

# Replace the first conv layer in the model with the new layer
mobilenet.features[0][0] = new_first_conv_layer

# Verify the changes by printing the model summary or a specific part
print(mobilenet.features[0])
print(mobilenet)


Conv2dNormActivation(
  (0): Conv2d(5, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  (2): Hardswish()
)
MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(5, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affi